# Daub 5/3 integer-to-integer system

In [12]:
import numpy as np
import matplotlib.pyplot as plt

Импорт пользовательских функций.

In [13]:
from Functions.daub_5_3 import daub_5_3

In [14]:
f = [1, -2, 6, 8]
N = len(f)
d = [0] * (N//2)
a = [0] * (N//2)

In [15]:
def indx_even(k,N):
    if k >= N:
        k = - (k - N + 2)
    elif k < 0:
        k = -k
    return k

In [16]:
def d_func(k,f,N):
    return f[indx_even(2*k+1,N)] - 1/2*(f[indx_even(2*k,N)] + f[indx_even(2*k+2,N)])

In [17]:
for k in range(N//2):
    d[k] = d_func(k,f,N)
    a[k] = -1/8*(f[indx_even(2*k-2,N)] + f[indx_even(2*k+2,N)]) + 1/4*(f[indx_even(2*k-1,N)] + f[indx_even(2*k+1,N)]) + 3/4*f[indx_even(2*k,N)]
print('a', a)
print('d', d)

a [-1.75, 5.125]
d [-5.5, 2.0]


In [18]:
for k in range(N//2):
    a[k] = f[indx_even(2*k,N)] + 1/4*(d_func(k-1,f,N) + d[k])
print('a', a)
print('d', d)

a [-1.75, 5.125]
d [-5.5, 2.0]


In [23]:
Mdir = np.matrix(np.concatenate(daub_5_3(N), axis = 0))
#Mrev = np.linalg.inv(Mdir).T
np.matrix(f)*(Mdir.T)

matrix([[-1.75 ,  5.125, -5.5  ,  2.   ]])

In [20]:
daub_5_3(N)

(array([[ 0.75 ,  0.5  , -0.25 ,  0.   ],
        [-0.125,  0.25 ,  0.625,  0.25 ]]),
 array([[-0.5,  1. , -0.5,  0. ],
        [ 0. ,  0. , -1. ,  1. ]]))